# dMRI Data Reconstruction


In this notebook, we will reconstruct MRI imgaes from raw data by using Python.This includes: 1. Data processing; 2. DTI reconstruction and 3. DKI reocnstruction.

## Data Preprocessing

Data preprocessing is quit important for dMRI reconstruction. Different data preprocessing may lead to different reconstruction image qualities, which will make the comparation of different reconstruct methods unreliable. Thus, here we first preprocessing MRI by following same steps: denosing, topup (susceptibility-induced distortion correction) and eddy current-induced distortion and motion correction.
![d](/home/erjun/githubEZ/dMRI_BHS/Fig/dwi_TOPUP_flowchart.png)

### __Import python libraries__

In [ ]:
import os #TO control directories
import nibabel as nib # read and save medical images
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import plotly
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage import io # image/video read and show

import timeit #compute time, useage: timeit.timeit()
import math
import time
import warnings
import numpy as np
import pandas as pd

import nipype.interfaces.fsl as fsl #topup
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces.fsl import Eddy
from nipype.testing import anatfile

from dipy.denoise.localpca import mppca #denoising
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.reconst.dti import fractional_anisotropy
from dipy.reconst.dti import color_fa
import dipy.reconst.dki as dki



from Extract_b0_Image import Extract_b0_Image
from nib_rdshow_img import nib_rdshow_img
from nib_read_img import nib_read_img
from nib_show_img import nib_show_img
from vol_plot import vol_plot
from interact_vol_plot import interact_vol_plot

# Import libararies used to data analysis
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

### Set data path and load data

In [ ]:
data_path = "/home/erjun/githubEZ/dMRI_BHS/dMRI_data/dwi3"
AP_file = 'Original_AP_dwi.nii.gz' # dMRI data
PA_file = 'Original_PA_dwi.nii.gz' 
bvals_AP_file = 'Original_AP_dwi.bval' # bval file
bvecs_AP_file = 'Original_AP_dwi.bvec' # bvec file
bvals_PA_file = 'Original_PA_dwi.bval' # bval file
bvecs_PA_file = 'Original_PA_dwi.bvec' # bvec file
denoised_AP_file = 'Denoised_AP_dwi.nii.gz' # output file after denoising
denoised_PA_file = 'Denoised_PA_dwi.nii.gz' # output file after denoising

cwdir = os.getcwd()
os.chdir(data_path) #directory setting
# Load images
img_AP = nib.load(os.path.join(data_path,AP_file)) # path.join connects data path given by
data_AP = img_AP.get_data()# I,age data array but without position information; need to be combine with image.affine (data position info)
img_PA = nib.load(os.path.join(data_path,PA_file)) # path.join connects data path given by
data_PA = img_PA.get_data()
# Use dipy to denoise
hdr=img_AP.header
print(hdr)

### Data type (NII.GZ)

- NII (NifTI, format by Neuroimaging Informatics Technology Initiative ) file is commonly used format for multi-dimensional (can be up to 7-dimensional) neuroimaging data. Fisrt four dimension: spatial dimensions and time. 

- GZ means gzip-compressed NII files.
* nib.nifil.Nifti1Image: three parts included in, namely, image data array, an affine array and image metadata.
* image metadata: machine info., voxel size and slices

Thus, in order to know the exact position of each voxel, we have to combine image data array and affine array. For more information, please check [fMRI Processing based on python](https://ff120.github.io/2016/06/12/%E8%AE%A4%E7%9F%A5%E7%A5%9E%E7%BB%8F%E7%A7%91%E5%AD%A6%E4%B8%93%E9%A2%98/%E4%BD%BF%E7%94%A8Python%E5%A4%84%E7%90%86fMRI%E6%95%B0%E6%8D%AE/).

### __Denoising__

Here we use [Marcenko-Pastur PCA algorithm](https://dipy.org/documentation/1.0.0./examples_built/denoise_mppca/) to denoise images. This algorithm has been shown to provide an optimal compromise between noise suppression and loss of anatomical information for different techniques such as DTI.

During the denoising, mppca use a 3D sliding window (decised by denoising radius, pathc_radius) to denoise. Basicaly, this 3D sliding window voxles should be larger than DTI volumes.

* Input: AP_file and PA_file, patch radius
* Output: 'Denoised_AP_dwi.nii.gz' and 'Denoised_AP_dwi.nii.gz'

In [ ]:
# Denoise and save positive phase-encoded direciton epi data
denoised_AP = mppca(data_AP, patch_radius=4) # If volume is about 67, pathc_radius can be set to 2
nib.save(nib.Nifti1Image(data_AP, img_AP.affine), os.path.join(data_path,denoised_AP_file))
# Denoise and save negative phase-encoded direciton epi data
denoised_PA = mppca(data_PA, patch_radius=4) # If volume is about 67, pathc_radius can be set to 2
nib.save(nib.Nifti1Image(data_PA, img_PA.affine), os.path.join(data_path,denoised_PA_file))

print('DONE')

#### Extract b0 images

* Input file: ap_file, or pa_file, or denoised_AP_file, or denoised_PA_file
* Corresponding extract 3D file

In [ ]:
# Set default output type and test ExtractROI tool for Define b_0 image extraction function
fsl.FSLCommand.set_default_output_type('NIFTI_GZ')
# This is used to test fsl.ExtractROI, if you see bar.nii.gz file in the data_path, it works well.
fslroi = fsl.ExtractROI(in_file=anatfile, roi_file='bar.nii.gz', t_min=0,t_size=1)
fslroi.cmdline == 'fslroi %s bar.nii.gz 0 1' % anatfile

In [ ]:
# Extract b0 images
# Extract_bo_Image is a self-define funtion
Extract_b0_Image(AP_file, 'Extract_AP_b0.nii.gz')
Extract_b0_Image(PA_file, 'Extract_PA_b0.nii.gz')
Extract_b0_Image(denoised_AP_file, 'Extract_denoised_AP_b0.nii.gz')
Extract_b0_Image(denoised_PA_file, 'Extract_denoised_PA_b0.nii.gz')

In [ ]:
# Check denoised AP files and images
img_Extract_AP_b0 = nib_read_img('Extract_AP_b0.nii.gz')
nib_show_img(img_Extract_AP_b0,28,80)

img_Extract_PA_b0_2=nib_read_img('Extract_denoised_AP_b0.nii.gz')
nib_show_img(img_Extract_PA_b0_2,28,80)

### Merge images

* Input files: 'Extract_denoised_AP_b0.nii.gz' +'Extract_denoised_PA_b0.nii.gz'
* Output files: 'Extract_denoised_AP_b0_merged.nii.gz', 'topup_encoding.txt'

While edit topup encoding file, phase encoded direciton can be found in .json file. The fourth parameter is the [time duration](file:///home/erjun/Downloads/topup(2f)ExampleTopupFollowedByApplytopup.html) between the readout of the centre of the first echo and the centre of the last echo. It can also be found in .json file or use parameters in .json file to calculate it. 

In [ ]:
# Use fslmerge to concatenate images
merger = fsl.Merge()
merger.inputs.in_files = ['Extract_denoised_AP_b0.nii.gz','Extract_denoised_PA_b0.nii.gz']
merger.inputs.dimension = 't'
merger.inputs.output_type = 'NIFTI_GZ'
#merger = fsl.Merge(in_files=['epi_b0.nii.gz','epi_rev_b0.nii.gz'],dimension = 't',output_type='NIFTI_GZ')
merger.run()

# Generate toup_encoding file
file = open('topup_encoding.txt','w')
file.write('0 -1 0 0.03124\n0 1 0 0.03124')# acquired with positive and negative phase-encode blips in y-direction
file.close()

### TOPUP
* Input files: 'Extract_denoised_AP_b0_merged.nii.gz' +'topup_encoding.txt'
* Output files: 'Extract_denoised_AP_b0_merged_base_fieldcoef.nii.gz', 'Extract_denoised_AP_b0_merged_base_movpar.txt', 'Extract_denoised_AP_b0_merged_corrected.nii.gz',Extract_denoised_AP_b0_merged_field.nii.gz, Extract_denoised_AP_b0_merged_topup.log, jac_01.nii.gz, jac_02.nii.gz, warpfield_01.nii.gz, warpfield_02.nii.gz, xfm_01.mat, xfm_02.mat

In [ ]:
topup = TOPUP()
topup.inputs.in_file = 'Extract_denoised_AP_b0_merged.nii.gz'
topup.inputs.encoding_file = "topup_encoding.txt"
topup.inputs.output_type = 'NIFTI_GZ'
topup.cmdline
topup.run()

### ApplyTOPUP
* Input files: 'Extract_denoised_AP_b0.nii.gz','Extract_denoised_PA_b0.nii.gz'
* Output files: Extract_denoised_AP_b0_merged_corrected.nii.gz

In [ ]:
applytopup = ApplyTOPUP()
applytopup.inputs.in_files = ['Extract_denoised_AP_b0.nii.gz','Extract_denoised_PA_b0.nii.gz']
applytopup.inputs.encoding_file = "topup_encoding.txt"
applytopup.inputs.in_topup_fieldcoef = "Extract_denoised_AP_b0_merged_base_fieldcoef.nii.gz"
applytopup.inputs.in_topup_movpar = "Extract_denoised_AP_b0_merged_base_movpar.txt"
applytopup.inputs.output_type = "NIFTI_GZ"
applytopup.cmdline
applytopup.run()        
print('DONE')

### bet

* Input files: 'Extract_denoised_AP_b0_corrected.nii.gz'
* Output files: bet_brain.nii.gz, bet_brain_mask.nii.gz, index.txt

In [ ]:
btr = fsl.BET(in_file= 'Extract_denoised_AP_b0_corrected.nii.gz',
              frac=0.2, out_file='bet_brain.nii.gz', mask=True)
btr.run()

# total nuber of volumes in dwi data
img_denoised_AP = nib.load(denoised_AP_file).get_data()
nvolumes = img_denoised_AP.shape[-1]

file = open('index.txt','w')
for i in range(0, nvolumes):
    file.write('1 ')
file.close()

# brain, brain_mask, index

### EDDY

During MRI scannning, subject movements and eddy current-induced distortions may occur. These distortion can be corrected by using FSL.Eddy. 
* Input files: denoised_AP_file, bet_brain_mask.nii.gz, index.txt
* Output files: eddy_corrected_AP.nii.gz

In [ ]:
eddy = fsl.Eddy(in_file = denoised_AP_file,
                in_mask  = 'bet_brain_mask.nii.gz',
                in_index = 'index.txt',
                in_acqp  = 'topup_encoding.txt',
                in_topup_fieldcoef = 'Extract_denoised_AP_b0_merged_base_fieldcoef.nii.gz',
                in_topup_movpar = 'Extract_denoised_AP_b0_merged_base_movpar.txt',
                in_bvec  = bvecs_AP_file,
                in_bval  = bvals_AP_file, use_cuda = False, 
                is_shelled=True)
eddy.run()

In [ ]:
# Extract_b0_Image('denoised_epi_b0_merged_corrected.nii.gz', 'denoised_epi_b0_merged_corrected_0.nii.gz')
#Extract_b0_Image('denoised_epi_b0_merged.nii.gz', 'denoised_epi_b0_merged_0.nii.gz')
Extract_b0_Image('eddy_corrected_AP.nii.gz','Extract_eddy_correct_AP.nii.gz')
Extract_b0_Image('Extract_denoised_AP_b0_merged.nii.gz','Extract_denoised_AP_b0_merged1.nii.gz')
Extract_b0_Image('Extract_denoised_AP_b0_merged_corrected.nii.gz','Extract_denoised_AP_b0_merged_corrected1.nii.gz')

T11=np.fliplr(nib_read_img('Extract_AP_b0.nii.gz'))
T12=np.fliplr(nib_read_img('Extract_PA_b0.nii.gz'))
T13=np.fliplr(nib_read_img('Extract_denoised_AP_b0.nii.gz'))
T14=np.fliplr(nib_read_img('Extract_denoised_PA_b0.nii.gz'))
T15=np.fliplr(nib_read_img('Extract_denoised_AP_b0_merged1.nii.gz'))
T16=np.fliplr(nib_read_img('Extract_denoised_AP_b0_merged_corrected1.nii.gz'))
T17=np.fliplr(nib_read_img('bet_brain.nii.gz'))
T18=np.fliplr(nib_read_img('bet_brain_mask.nii.gz'))
T19=np.fliplr(nib_read_img('Extract_eddy_correct_AP.nii.gz'))


fig, ([ax1, ax2,ax3],[ax4, ax5,ax6],[ax7, ax8,ax9]) = plt.subplots(3,3,figsize=(15,12),subplot_kw={'xticks': [], 'yticks': []})
ax1.imshow(T11[:,:,28].T); ax1.set_title('Original_AP_Image',fontweight='bold',size=13)
ax2.imshow(T12[:,:,28].T); ax2.set_title('Original_PA_Image',fontweight='bold',size=13)
ax3.imshow(T13[:,:,28].T); ax3.set_title('Denoised_AP_Image',fontweight='bold',size=13)
ax4.imshow(T14[:,:,28].T); ax4.set_title('Denoised_PA_Image',fontweight='bold',size=13)
ax5.imshow(T15[:,:,28].T); ax5.set_title('Merged_Image',fontweight='bold',size=13)
ax6.imshow(T16[:,:,28].T); ax6.set_title('TOPUP_AP_Image',fontweight='bold',size=13)
ax7.imshow(T17[:,:,28].T); ax7.set_title('Brain_After_bet',fontweight='bold',size=13)
ax8.imshow(T18[:,:,28].T); ax8.set_title('Brain_mask',fontweight='bold',size=13)
ax9.imshow(T19[:,:,28].T); ax9.set_title('Eddy_Correct_AP_Image',fontweight='bold',size=13)

fig, ([ax1, ax3,ax6],[ax9, ax8,ax7]) = plt.subplots(2,3,figsize=(15,12),subplot_kw={'xticks': [], 'yticks': []})
ax1.imshow(T11[:,:,28].T); ax1.set_title('Original_AP_Image',fontweight='bold',size=13)
#ax2.imshow(T12[:,:,28].T); ax2.set_title('Original_PA_Image',fontweight='bold',size=13)
ax3.imshow(T13[:,:,28].T); ax3.set_title('Denoised_AP_Image',fontweight='bold',size=13)
#ax4.imshow(T14[:,:,28].T); ax4.set_title('Denoised_PA_Image',fontweight='bold',size=13)
#ax5.imshow(T15[:,:,28].T); ax5.set_title('Merged_Image',fontweight='bold',size=13)
ax6.imshow(T16[:,:,28].T); ax6.set_title('TOPUP_AP_Image',fontweight='bold',size=13)
ax7.imshow(T17[:,:,28].T); ax7.set_title('Brain_After_bet',fontweight='bold',size=13)
ax8.imshow(T18[:,:,28].T); ax8.set_title('Brain_mask',fontweight='bold',size=13)
ax9.imshow(T19[:,:,28].T); ax9.set_title('Eddy_Correct_AP_Image',fontweight='bold',size=13)

## DTI Reconstruction

In [ ]:
# Load data files
img1 = nib.load(os.path.join(data_path,'eddy_corrected_AP.nii.gz'))
data = img1.get_data()

img2 = nib.load(os.path.join(data_path,'bet_brain_mask.nii.gz'))
brainmask = img2.get_data()

bvals, bvecs = read_bvals_bvecs(os.path.join(bvals_AP_file),
                                os.path.join(data_path,bvecs_AP_file))
bTable= gradient_table(bvals, bvecs)

In [ ]:
# DTI model
ten_model = TensorModel(bTable)
ten_fit = ten_model.fit(data, brainmask)
        
# Save DTI parametric maps
if not os.path.exists(data_path+'/DTI/'):
    os.mkdir(data_path+'/DTI')
output_path = data_path+'/DTI/'
        
DTI_FA = ten_fit.fa
DTI_AD = ten_fit.ad
DTI_RD = ten_fit.rd
DTI_MD = ten_fit.md
        
nib.save(nib.Nifti1Image(DTI_FA, img1.affine), os.path.join(output_path,'FA.nii.gz'))
nib.save(nib.Nifti1Image(DTI_MD, img1.affine), os.path.join(output_path,'MD.nii.gz'))
nib.save(nib.Nifti1Image(DTI_RD, img1.affine), os.path.join(output_path,'RD.nii.gz'))
nib.save(nib.Nifti1Image(DTI_AD, img1.affine), os.path.join(output_path,'AD.nii.gz'))
    
#Save FA RGB map
fa = fractional_anisotropy(ten_fit.evals)
cfa = color_fa(fa, ten_fit.evecs)
DTI_FA = np.clip(fa, 0, 1)
DTI_RGB = color_fa(fa, ten_fit.evecs)

nib.save(nib.Nifti1Image(np.array(255 * cfa, 'uint8'), img1.affine), os.path.join(output_path,'FA_RGB.nii.gz'))

print('Done!')

In [ ]:
# DKI MODEL
dkimodel = dki.DiffusionKurtosisModel(bTable)
dkifit = dkimodel.fit(data, brainmask)
        
# Save DKI parametric maps
if not os.path.exists(data_path+'/DKI/'):
    os.mkdir(data_path+'/DKI')
data_path_saveImage = data_path+'/DKI/'
        
DKI_FA = dkifit.fa
DKI_MD = dkifit.md
DKI_RD = dkifit.rd
DKI_AD = dkifit.ad

DKI_MK = dkifit.mk(0, 3)
DKI_AK = dkifit.ak(0, 3)
DKI_RK = dkifit.rk(0, 3)
        
nib.save(nib.Nifti1Image(DKI_FA, img1.affine), os.path.join(data_path_saveImage,'dki_FA.nii.gz'))
nib.save(nib.Nifti1Image(DKI_MD, img1.affine), os.path.join(data_path_saveImage,'dki_MD.nii.gz'))
nib.save(nib.Nifti1Image(DKI_RD, img1.affine), os.path.join(data_path_saveImage,'dki_RD.nii.gz'))
nib.save(nib.Nifti1Image(DKI_AD, img1.affine), os.path.join(data_path_saveImage,'dki_AD.nii.gz'))
        
nib.save(nib.Nifti1Image(DKI_AK, img1.affine), os.path.join(data_path_saveImage,'AK.nii.gz'))
nib.save(nib.Nifti1Image(DKI_RK, img1.affine), os.path.join(data_path_saveImage,'RK.nii.gz'))
nib.save(nib.Nifti1Image(DKI_MK, img1.affine), os.path.join(data_path_saveImage,'MK.nii.gz'))
        
print('DONE!')

## Data Visualization

In this section, I first show the basic images generate during preprocessing and final image reconstruction process. Then I will go the data visualization part. And before that, to create images more eassily, I will definie several image showing function first.

All the images are generated from this project and base on the data preprocessing and iamge reconstruction part.

Let's go to check what basic images we aready have!

### Basic Images

In [ ]:
# DTI Images
# set plot background
#plt.style.use('seaborn-dark')
plt.style.use('grayscale')
# plot paramter maps  
axf1=np.fliplr(DTI_RGB[:,:,28,:])
#axf1=axf1.T
axf2=np.fliplr(DTI_MD[:,:,28])
axf2=axf2.T
axf3=np.fliplr(DTI_RD[:,:,28])
axf3=axf3.T
axf4=np.fliplr(DTI_AD[:,:,28])
axf4=axf4.T
fig, [ax0,ax2, ax3, ax4] = plt.subplots(1,4,figsize=(12,10),subplot_kw={'xticks': [], 'yticks': []})
ax0.imshow(axf1); ax0.set_title('Color coded FA',fontweight='bold',size=13)
#ax1.imshow(DTI_FA[:,:,28]); ax1.set_title('Fractional anisotropy',fontweight='bold',size=13)
ax2.imshow(axf2); ax2.set_title('Mean diffusivity',fontweight='bold',size=13)
ax3.imshow(axf3); ax3.set_title('Radial diffusivity',fontweight='bold',size=13)
ax4.imshow(axf4); ax4.set_title('Axial diffusivity',fontweight='bold',size=13)

In [ ]:
# DKI images  
axt0=np.fliplr(DKI_AD[:,:,28])
axt0=axt0.T

axt1=np.fliplr(DKI_RD[:,:,28])
axt1=axt1.T

axt2=np.fliplr(DKI_MD[:,:,28])
axt2=axt2.T

axt3=np.fliplr(DKI_AK[:,:,28])
axt3=axt3.T

axt4=np.fliplr(DKI_RK[:,:,28])
axt4=axt4.T

axt5=np.fliplr(DKI_MK[:,:,28])
axt5=axt5.T

fig, ([ax0, ax1, ax2],[ax3, ax4, ax5]) = plt.subplots(2,3,figsize=(10,8),subplot_kw={'xticks': [], 'yticks': []})
ax0.imshow(axt0); ax0.set_title('Axial diffusivity',fontweight='bold',size=10)
ax1.imshow(axt0); ax1.set_title('Radial diffusivity',fontweight='bold',size=10)
ax2.imshow(axt0); ax2.set_title('Mean diffusivity',fontweight='bold',size=10)
ax3.imshow(axt0); ax3.set_title('Axial kurtosis',fontweight='bold',size=10)
ax4.imshow(axt0); ax4.set_title('Radial kurtosis',fontweight='bold',size=10)
ax5.imshow(axt0); ax5.set_title('Mean kurtosis',fontweight='bold',size=10)

### Visualization

In [ ]:
# Function, used to show images, definition
# To read images data from nii.gz file: nib_read_img(path)
# To show images from nii.gz file: nib_show_img(img0,slices,intenseScale)
# To read and show images from nii.gz file: nib_rdshow_img(Images,Slices,IntenseScale,TitleImg)
# Function to show volume slices images: vol_plot(x) and interact_vol_plot(x,IntenseScale):

#### 3D volume slice image

This images will quickly check the general apperance of you images during the slice-by-slice animation. One can also rotate it and zoom in to see clearly.

Here, as an example, I just show FA map generated from DTI model. Once can change it to other maps, such as DTI_MD or DKI_FA.

In [ ]:
#vol_plot(DTI_FA[:,:,:]) # or can use interact_vol_plot(DTI_MD,90) to show images
interact_vol_plot(DTI_FA,0.90)

#### Data visualization (Preprocessing part)

I am curious about what changed of the iamges during our data preprocessing. This interactive images can be used to reach this purpose.

One can control Slices to check different slice changes among them. You can control the level of high color scale to get higher image contrast. You can put cursor on images to get the exact color value if you want.

In [ ]:
# Set image data for visualization (preprocessing part)
# This will be used for visualization of preprocessing part 1 and part 2
F11='Extract_AP_b0.nii.gz'
F12='Extract_PA_b0.nii.gz'
F13='Extract_denoised_AP_b0.nii.gz'
F14='Extract_denoised_PA_b0.nii.gz'
F15='Extract_denoised_AP_b0_merged1.nii.gz'
F16='Extract_denoised_AP_b0_merged_corrected1.nii.gz'
F17='bet_brain.nii.gz'
F18='bet_brain_mask.nii.gz'
F19='Extract_eddy_correct_AP.nii.gz'


PreproImage= dict()
PreproImage['Original_AP_Image']=F11
PreproImage['Original_PA_Image']=F12
PreproImage['Denoised_AP_Image']=F13
PreproImage['Denoised_PA_Image']=F14
PreproImage['Merged_Image']=F15
PreproImage['TOPUP_AP_Image']=F16
PreproImage['Brain_After_bet']=F17

PreproImage['Brain_mask']=F18
PreproImage['Eddy_Correct_AP_Image']=F19

# Load DTI and DKI results
# 
Images= dict()
Images['DTI_FA_RGB']=(data_path+'/DTI/FA_RGB.nii.gz')
Images['DTI_AD']=(data_path+'/DTI/AD.nii.gz')
Images['DTI_FA']=data_path+'/DTI/FA.nii.gz'
Images['DTI_MD']=data_path+'/DTI/MD.nii.gz'
Images['DTI_RD']=data_path+'/DTI/RD.nii.gz'
# DKI
Images['DKI_AD']=data_path+'/DKI/dki_AD.nii.gz'
Images['DKI_FA']=data_path+'/DKI/dki_FA.nii.gz'
Images['DKI_MD']=data_path+'/DKI/dki_MD.nii.gz'
Images['DKI_RD']=data_path+'/DKI/dki_RD.nii.gz'
Images['DKI_AK']=data_path+'/DKI/AK.nii.gz'
Images['DKI_MK']=data_path+'/DKI/MK.nii.gz'
Images['DKI_RK']=data_path+'/DKI/RK.nii.gz'

In [ ]:
# Preprocessing image visualization 1
interactive(nib_rdshow_img,Images=PreproImage,Slices=widgets.IntSlider(min=0,max=43,step=1,value=28),\
         IntenseScale=widgets.IntSlider(min=0,max=100,step=1,value=90),\
           TitleImg='Reconstructed Images of Different Models')

In [ ]:
# Preprocessing image visualization 2
# Function below is used to show correct figure title while animaiton playing
def nib_rdshow_play(Slices,IntenseScale,NoX):
    warnings.filterwarnings('ignore')
    if(NoX==0):
        Image=F11
        TitleImg='Original_AP_Image'
    elif(NoX==1):
        Image=F12 #'epi_b0_merged.nii.gz'
        TitleImg='Original_PA_Image'
    elif(NoX==2):
        Image=F13
        TitleImg='Denoised_AP_Image'
    elif(NoX==3):
        Image=F14
        TitleImg='Denoised_PA_Image'
    elif(NoX==4):
        Image=F15 #'epi_b0_merged.nii.gz'
        TitleImg='Merged_Image'
    elif(NoX==5):
        Image=F16
        TitleImg='TOPUP_AP_Image'
    elif(NoX==6):
        Image=F17
        TitleImg='Brain_After_bet'
    elif(NoX==7):
        Image=F18
        TitleImg='Brain_mask'
    else:
        Image=F15
        TitleImg='Eddy_Correct_AP_Image'
    image_data = nib.load(Image).get_data()
    img00=image_data[:,:,Slices]
    zmax0=img00.max()
    fig=px.imshow(image_data[:,:,Slices],color_continuous_scale="Viridis",\
                  zmin=0,zmax=zmax0*IntenseScale/100,\
                  labels={},template="plotly_white")
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_layout(title=TitleImg)
    fig.show()

interactive(nib_rdshow_play,Slices=widgets.IntSlider(min=0,max=43,step=1,value=28),\
         IntenseScale=widgets.IntSlider(min=0,max=100,step=1,value=90),\
           NoX = widgets.Play(value=0,min=0,max=8,step=1,interval=2000,description="Press play",\
                                   disabled=False))

#### Data visualization (Reconstruction part)

This part is used to compare images generated from DTI and DKI models. The founction is almost the same as virsualizaiotn above. But to make us better choose iamges, we use dropdown bar to replace slider animation.

In [ ]:
# Reconstruction image visualization 1
interactive(nib_rdshow_img,Images=Images,Slices=widgets.IntSlider(min=0,max=43,step=1,value=22),\
         IntenseScale=widgets.IntSlider(min=0,max=100,step=1,value=90),\
           TitleImg='Reconstructed Images of Different Models')

## Data Analysis

In [ ]:
# Import libararies used to data analysis
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
shapeT18=np.shape(T18)
data_MR=np.zeros([80,4])
print('MR data shape:',shapeT18)
print('dMRI data shape:',np.shape(data_MR))

In [ ]:
# Generate data of right heimesphere and left heimesphere
for i in range(0,39):
    data_MR[i,0]=i
    data_MR[i+39,0]=i
    data_MR[i,3]=0
    data_MR[i+39,3]=1
    
    EDDY_valueA=T19[0:47,:,i]
    EDDY_valueA=np.sum(EDDY_valueA)
    data_MR[i,2]=5*np.log2(1+EDDY_valueA)
    EDDY_valueB=T19[48:95,:,i]
    EDDY_valueB=5*np.sum(EDDY_valueB)
    data_MR[i+39,2]=np.log2(1+EDDY_valueB)
    
    Num_voxA=T18[0:47,:,i]
    Num_voxA=np.sum(Num_voxA)
    data_MR[i,1]=Num_voxA
    Num_voxB=T18[48:95,:,i]
    Num_voxB=np.sum(Num_voxB)
    data_MR[i+39,1]=Num_voxB
np.savetxt('brain_water.csv',data_MR,delimiter=',')
brain_water_csv = pd.read_csv('brain_water.csv',names=['Num_slice','Num_vox','Semi_AB','Position'])

In [ ]:
# Details of data waiting for analyzing
brain_water_csv=brain_water_csv.dropna() # Delete NAN data line
brain_water_csv.head()

In [ ]:
# Details of data waiting for analyzing
brain = brain_water_csv.describe()
brain

In [ ]:
#sns.pairplot(brain_water_csv)

In [ ]:
# Checek frequency of right and left hemispheres
brain_water_csv['Position'].plot.hist()

In [ ]:
# Check brain diffusion water data location
brain_water_csv['Semi_AB'].plot.density()

In [ ]:
# Set train dataset (catergoricalfetures and target class)
x1=(brain_water_csv['Num_slice'])
x2=(brain_water_csv['Num_vox'])
x3=(brain_water_csv['Semi_AB'])
x=np.vstack((x1,x2,x3))
x=x.T
y=5*(brain_water_csv['Position'])
y=y.T

In [ ]:
# Split dataset into train dataset and test dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

In [ ]:
# Use KNN with different K value to train and test
error_rate = []
for i in range(1,40):    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    pred_i = knn.predict(x_test)
    error_rate.append(np.mean(pred_i != y_test))

plt.figure(figsize=(12,9))
plt.plot(range(1,40),error_rate,color='gray', linestyle='dashed', marker='o',
         markerfacecolor='black', markersize=5)
plt.title('Error Rate vs. K Value', fontsize=20)
plt.xlabel('K value',fontsize=14)
plt.ylabel('Error Rate',fontsize=14)

In [ ]:
# Use KNN to train and test
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
pred = knn.predict(x_test)
conf_mat=confusion_matrix(y_test,pred)
print('Conf_mat',conf_mat)

# Show clasification report
print(classification_report(y_test,pred))
print("Misclassification error rate:",round(np.mean(pred!=y_test),3))